In [9]:
# import kagglehub

# # Download dataset dari Kaggle
# path = kagglehub.dataset_download("canggih/indonesian-food-recipes")

# print("Path to dataset files:", path)

In [10]:
import pandas as pd
import glob
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Tentukan path ke folder yang berisi file CSV
path = 'C:/Users/ACER/.cache/kagglehub/datasets/canggih/indonesian-food-recipes/versions/3/*.csv'  # Ganti dengan path yang sesuai

# Mengambil semua file CSV
all_files = glob.glob(path)

# Membaca dan menggabungkan semua file CSV menjadi satu DataFrame
df_list = [pd.read_csv(file) for file in all_files]
indonesia_food_dataset = pd.concat(df_list, ignore_index=True)

# Tampilkan beberapa baris pertama untuk memverifikasi
print(indonesia_food_dataset.head())


                      Title  \
0          Ayam Woku Manado   
1  Ayam goreng tulang lunak   
2          Ayam cabai kawin   
3               Ayam Geprek   
4               Minyak Ayam   

                                         Ingredients  \
0  1 Ekor Ayam Kampung (potong 12)--2 Buah Jeruk ...   
1  1 kg ayam (dipotong sesuai selera jangan kecil...   
2  1/4 kg ayam--3 buah cabai hijau besar--7 buah ...   
3  250 gr daging ayam (saya pakai fillet)--Secuku...   
4  400 gr kulit ayam & lemaknya--8 siung bawang p...   

                                               Steps  Loves  \
0  Cuci bersih ayam dan tiriskan. Lalu peras jeru...      1   
1  Haluskan bumbu2nya (BaPut, ketumbar, kemiri, k...      1   
2  Panaskan minyak di dalam wajan. Setelah minyak...      2   
3  Goreng ayam seperti ayam krispi--Ulek semua ba...     10   
4  Cuci bersih kulit ayam. Sisihkan--Ambil 50 ml ...      4   

                                          URL  
0          /id/resep/4473027-ayam-woku-manado  
1

In [11]:
indonesia_food_dataset.to_csv("Indonesia_food_dataset.csv", index=False)


In [12]:
# Mengganti nilai NaN dengan string kosong
indonesia_food_dataset['Ingredients'] = indonesia_food_dataset['Ingredients'].fillna('')

print(indonesia_food_dataset.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15641 entries, 0 to 15640
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        15641 non-null  object
 1   Ingredients  15641 non-null  object
 2   Steps        15595 non-null  object
 3   Loves        15641 non-null  int64 
 4   URL          15641 non-null  object
dtypes: int64(1), object(4)
memory usage: 611.1+ KB
None


In [13]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# Buat TF-IDF Vectorizer untuk kolom 'Ingredients'
tfidf = TfidfVectorizer()

# Fit dan transform kolom 'Ingredients' menjadi matriks TF-IDF
tfidf_matrix = tfidf.fit_transform(indonesia_food_dataset['Ingredients'])


In [14]:

def recommend_by_ingredients(ingredients_input):
    # Gabungkan 3 bahan input menjadi satu string
    input_str = " ".join(ingredients_input)

    # Transform input bahan menjadi vektor TF-IDF
    input_tfidf = tfidf.transform([input_str])

    # Hitung cosine similarity antara input dan semua resep dalam dataset
    sim_scores = cosine_similarity(input_tfidf, tfidf_matrix).flatten()

    # Urutkan resep berdasarkan skor kemiripan
    sim_scores = list(enumerate(sim_scores))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Ambil indeks resep yang paling mirip (5 teratas)
    sim_scores = sim_scores[:3]

    # Dapatkan indeks resep
    recipe_indices = [i[0] for i in sim_scores if i[1] > 0]  # Hanya ambil yang memiliki skor > 0

    if not recipe_indices:  # Jika tidak ada indeks yang ditemukan
        return "Tidak ada resep yang ditemukan berdasarkan bahan yang dimasukkan."

    # Mengambil kolom yang diinginkan
    recommended_recipes = indonesia_food_dataset[['Title', 'Ingredients', 'Steps']].iloc[recipe_indices]

    # Format output
    output = []
    for index, row in recommended_recipes.iterrows():
        # Memisahkan langkah berdasarkan '--' dan menghapus elemen kosong
        steps = [step.strip() for step in row['Steps'].split('--') if step.strip()]

        # Menambahkan nomor langkah
        numbered_steps = [f"{i + 1}. {step}" for i, step in enumerate(steps)]

        steps_formatted = "\n".join(numbered_steps)  # Menggabungkan kembali langkah-langkah yang sudah dinomori

        # Menambahkan nomor pada setiap bahan di Ingredients
        ingredients_list = [ingredient.strip() for ingredient in row['Ingredients'].split('--') if ingredient.strip()]
        numbered_ingredients = [f"{i + 1}. {ingredient}" for i, ingredient in enumerate(ingredients_list)]

        ingredients_formatted = "\n".join(numbered_ingredients)  # Menggabungkan kembali bahan-bahan yang sudah dinomori

        output.append(f"**{row['Title']}**\n\n"
                      f"**Ingredients:**\n{ingredients_formatted}\n\n"
                      f"**Steps:**\n{steps_formatted}\n"
                      "-----------------------------------------\n")

    return "\n".join(output)


In [17]:
# Contoh input: bahan "ayam", "bawang putih", dan "kecap"
ingredients_input = ["ayam", "bawang", "saus"]
recommended_recipes = recommend_by_ingredients(ingredients_input)

# Tampilkan hasil rekomendasi
print(recommended_recipes)


**Orak arik tekor (telur kornet)**

**Ingredients:**
1. 1 sdm kornet sapi
2. 1 btr telur ayam
3. 2 siung bawang putih
4. 1 sdt saus tiram
5. 1 sdm saus tomat
6. 3 sdm saus pedas
7. secukupnya Gula
8. secukupnya Garam
9. Minyak goreng

**Steps:**
1. Cincang bawang putih tumis sampai harum. Masukkan kornet aduk" sampai kecoklatan. Setelah itu masukan telur di orak arik.
2. Tambahkan saus"an gula dan garam. Aduk smpai tercampur. Tes rasa dan siap di hidangkan bersama nasi hangat 😉
-----------------------------------------

**Kering Tempe Tahu (tempe/tahu sisa)**

**Ingredients:**
1. Tempe
2. Tahu
3. Cabe
4. Bawang merah
5. Bawang putih
6. Tomat
7. Kecap
8. Saus tiram
9. Saus tomat

**Steps:**
1. Potong-potong tempe dan tahu lalu goreng
2. Iris cabe, bawang putih (biasanya saya geprek) bawang merah dan tomat lalu tumis dengan sedikit minyak
3. Masukkan tempe dan tahu dalam tumisan, tambahkan kecap (ato kalo ada sisa sambel kecap bisa juga dipakai), saus tiram, saus tomat.
4. Angkat dan sia

In [16]:
import joblib
joblib.dump(tfidf,'vectorizer.pkl')
joblib.dump(recommended_recipes,'resep_model.pkl')


['resep_model.pkl']